In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir(r'/content/gdrive/My Drive/License_Plate/Plate_detect_and_recognize')

Mounted at /content/gdrive


In [ ]:
# remove warning message
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# required library
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from local_utils import detect_lp
from os.path import splitext,basename
from keras.models import model_from_json
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import LabelEncoder
import glob

### Part 1: Extract license plate from sample image

In [ ]:
def load_model(path):
    try:
        path = splitext(path)[0]
        with open('%s.json' % path, 'r') as json_file:
            model_json = json_file.read()
        model = model_from_json(model_json, custom_objects={})
        model.load_weights('%s.h5' % path)
        print("Loading model successfully...")
        return model
    except Exception as e:
        print(e)

In [ ]:
wpod_net_path = "wpod-net.json"
wpod_net = load_model(wpod_net_path)

Loading model successfully...


In [ ]:
def preprocess_image(image_path,resize=False):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255
    if resize:
        img = cv2.resize(img, (224,224))
    return img

def get_plate(image_path, Dmax=608, Dmin = 608):
    vehicle = preprocess_image(image_path)
    ratio = float(max(vehicle.shape[:2])) / min(vehicle.shape[:2])
    side = int(ratio * Dmin)
    bound_dim = min(side, Dmax)
    _ , LpImg, _, cor = detect_lp(wpod_net, vehicle, bound_dim, lp_threshold=0.5)
    return vehicle, LpImg, cor


# Create sort_contours() function to grab the contour of each digit from left to right
def sort_contours(cnts,reverse = False):
    i = 0
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
                                        key=lambda b: b[1][i], reverse=reverse))
    return cnts


# Load model architecture, weight and labels
json_file = open('MobileNets_character_recognition.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("License_character_recognition_weight.h5")
print("[INFO] Model loaded successfully...")

labels = LabelEncoder()
labels.classes_ = np.load('license_character_classes.npy')
print("[INFO] Labels loaded successfully...")


# pre-processing input images and pedict with model
def predict_from_model(image,model,labels):
    image = cv2.resize(image,(80,80))
    image = np.stack((image,)*3, axis=-1)
    prediction = labels.inverse_transform([np.argmax(model.predict(image[np.newaxis,:]))])
    return prediction




test_image_path = "../CLPD/CLPD/CLPD_1200/0.jpg"

vehicle, LpImg, cor = get_plate(test_image_path)

all_path = glob.glob("../CLPD/CLPD/CLPD_1200/*.jpg")
for test_image_path in all_path:
# fig = plt.figure(figsize=(12,6))
# grid = gridspec.GridSpec(ncols=2,nrows=1,figure=fig)
# fig.add_subplot(grid[0])
# plt.axis(False)
# plt.imshow(vehicle)
# grid = gridspec.GridSpec(ncols=2,nrows=1,figure=fig)
# fig.add_subplot(grid[1])
# plt.axis(False)
# plt.imshow(LpImg[0])


  try:
    vehicle, LpImg, cor = get_plate(test_image_path)
    if (len(LpImg)): #check if there is at least one license image
      # Scales, calculates absolute values, and converts the result to 8-bit.
      plate_image = cv2.convertScaleAbs(LpImg[0], alpha=(255.0))
    
    # convert to grayscale and blur the image
      gray = cv2.cvtColor(plate_image, cv2.COLOR_BGR2GRAY)
      blur = cv2.GaussianBlur(gray,(7,7),0)
    
    # Applied inversed thresh_binary 
      binary = cv2.threshold(blur, 180, 255,
                           cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    
      kernel3 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
      thre_mor = cv2.morphologyEx(binary, cv2.MORPH_DILATE, kernel3)

# # visualize results    
# fig = plt.figure(figsize=(12,7))
# plt.rcParams.update({"font.size":18})
# grid = gridspec.GridSpec(ncols=2,nrows=3,figure = fig)
# plot_image = [plate_image, gray, blur, binary,thre_mor]
# plot_name = ["plate_image","gray","blur","binary","dilation"]

# for i in range(len(plot_image)):
#     fig.add_subplot(grid[i])
#     plt.axis(False)
#     plt.title(plot_name[i])
#     if i ==0:
#         plt.imshow(plot_image[i])
#     else:
#         plt.imshow(plot_image[i],cmap="gray")

# # plt.savefig("threshding.png", dpi=300)



#set a thresh
    thresh = 100
    r, t = cv2.threshold(gray, thresh, 255, cv2.THRESH_BINARY)

# for binary in binarys:
#   c, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#   conts.append(c)

    cont, _ = cv2.findContours(t, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# creat a copy version "test_roi" of plat_image to draw bounding box
    test_roi = plate_image.copy()



# Initialize a list which will be used to append charater image
    crop_characters = []

    digit_w, digit_h = 30, 60


    for c in sort_contours(cont):
        (x, y, w, h) = cv2.boundingRect(c)
        ratio = h/w
        if 0.5<=ratio<=5: # Only select contour with defined ratio
            if h/plate_image.shape[0]>=0.5: # Select contour which has the height larger than 50% of the plate
                # Draw bounding box arroung digit number
                cv2.rectangle(test_roi, (x, y), (x + w, y + h), (0, 255,0), 2)

                # Sperate number and gibe prediction
                curr_num = thre_mor[y:y+h,x:x+w]
                curr_num = cv2.resize(curr_num, dsize=(digit_w, digit_h))
                _, curr_num = cv2.threshold(curr_num, 220, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                crop_characters.append(curr_num)

    print("Detect {} letters...".format(len(crop_characters)))
# fig = plt.figure(figsize=(10,6))
# plt.axis(False)
# plt.imshow(test_roi)
# plt.savefig('grab_digit_contour.png',dpi=300)





    fig = plt.figure(figsize=(15,3))
    cols = len(crop_characters)
    grid = gridspec.GridSpec(ncols=cols,nrows=1,figure=fig)

    final_string = ''
    for i,character in enumerate(crop_characters):
        # fig.add_subplot(grid[i])
        title = np.array2string(predict_from_model(character,model,labels))
        # plt.title('{}'.format(title.strip("'[]"),fontsize=20))
        final_string+=title.strip("'[]")
    # plt.axis(False)
    # plt.imshow(character,cmap='gray')

    print(final_string)


  except:
    print(test_image_path)

#plt.savefig('final_result.png', dpi=300)


[INFO] Model loaded successfully...
[INFO] Labels loaded successfully...
Detect 7 letters...
BWBYD69
Detect 9 letters...
B3DDU52T0
Detect 1 letters...
W
Detect 8 letters...
WWBBB303
Detect 0 letters...

../CLPD/CLPD/CLPD_1200/146.jpg
Detect 1 letters...
W
Detect 7 letters...
BB2BSBU
Detect 0 letters...

Detect 0 letters...

Detect 3 letters...
WB6
Detect 5 letters...
06256
Detect 0 letters...

Detect 6 letters...
BSWBBB
Detect 9 letters...
DD0S55555
Detect 0 letters...

Detect 1 letters...
W
Detect 0 letters...

Detect 6 letters...
EGSN6H
Detect 0 letters...

Detect 0 letters...

Detect 0 letters...



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:149: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Detect 1 letters...
T
Detect 4 letters...
0666
Detect 1 letters...
W
Detect 0 letters...

../CLPD/CLPD/CLPD_1200/1094.jpg
Detect 9 letters...
EBBD309B3
Detect 0 letters...

Detect 0 letters...

Detect 0 letters...

Detect 0 letters...

Detect 8 letters...
WWBW202S
Detect 0 letters...

Detect 0 letters...

Detect 0 letters...

Detect 7 letters...
EB5000B
Detect 7 letters...
E3B6666
Detect 0 letters...

Detect 3 letters...
3VU
Detect 7 letters...
33UD550
Detect 0 letters...

Detect 5 letters...
9B02U
Detect 3 letters...
BED
Detect 0 letters...

Detect 1 letters...
W
Detect 0 letters...

Detect 0 letters...

Detect 1 letters...
W
Detect 0 letters...

Detect 2 letters...
YN
Detect 5 letters...
BBBBB
Detect 7 letters...
BB6P295
Detect 0 letters...

Detect 7 letters...
EHUUUUZ
../CLPD/CLPD/CLPD_1200/1102.jpg
Detect 7 letters...
3B922B2
../CLPD/CLPD/CLPD_1200/1167.jpg
Detect 1 letters...
B
Detect 6 letters...
925BD0
Detect 2 letters...
I3
Detect 5 letters...
3B456
Detect 5 letters...
U2259
De

<Figure size 1080x216 with 0 Axes>

<Figure size 1080x216 with 0 Axes>

...

<Figure size 1080x216 with 0 Axes>

<Figure size 1080x216 with 0 Axes>

<Figure size 1080x216 with 0 Axes>

<Figure size 1080x216 with 0 Axes>

<Figure size 1080x216 with 0 Axes>